# 🇨🇿 Czech Character-Level GPT Evaluation

## Model Loading

In [1]:
CHECKPOINT_PATH = "checkpoint_250chinchilla" 
TOKENIZER_PATH = "checkpoint_250chinchilla/tokenizer.json" 

In [2]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM
from tokenizers import Tokenizer
from tqdm.auto import tqdm
import textwrap
import re

In [3]:
from model import CzechGPTConfig, CzechGPTModel

In [4]:
AutoConfig.register("czech-gpt", CzechGPTConfig)
AutoModelForCausalLM.register(CzechGPTConfig, CzechGPTModel)

In [5]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda:1


In [6]:
model = AutoModelForCausalLM.from_pretrained(
    CHECKPOINT_PATH, 
    trust_remote_code=True
)
model.to(device)
model.eval()

CzechGPTModel(
  (embed): ModuleDict(
    (wte): Embedding(64, 384)
    (drop): Dropout(p=0.1, inplace=False)
  )
  (body): ModuleDict(
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=True)
          (c_proj): Linear(in_features=384, out_features=384, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
          (qk_norm): QKNorm(
            (q_norm): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
            (k_norm): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
          )
          (rope): RoPE()
        )
        (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=True)
          (c_proj): Linear(in_features=1536, out_features=384,

In [7]:
tokenizer = Tokenizer.from_file(TOKENIZER_PATH)
pad_token_id = tokenizer.get_vocab().get("[PAD]", 0)

In [8]:
CONTEXT_LENGTH = model.config.context_length

## Helpers

In [9]:
def print_wrapped(text):
    print(textwrap.fill(text, width=100))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[^a-záčďéěíňóřšťúůýž0-9\s.,!?-]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def generate_from_prompt(prompt, max_new_chars=80, temperature=0.7, top_k=50):
    cleaned_prompt = clean_text(prompt)
    
    if len(cleaned_prompt) > CONTEXT_LENGTH:
        print(f"Warning: Cleaned prompt is too long ({len(cleaned_prompt)} > {CONTEXT_LENGTH} chars). Truncating.")
        cleaned_prompt = cleaned_prompt[-CONTEXT_LENGTH:]
    token_ids = tokenizer.encode(cleaned_prompt).ids
    input_ids = torch.tensor([token_ids], dtype=torch.long, device=device)
    output_ids = model.generate(
        input_ids, max_new_tokens=max_new_chars, temperature=temperature,
        top_k=top_k, do_sample=True, pad_token_id=pad_token_id
    )
    full_output = tokenizer.decode(output_ids[0].tolist(), skip_special_tokens=True)
    return full_output


## Part 1: General Text Generation

In [10]:
general_prompts = [
    "Byl jednou jeden starý hrad. V noci se z něj ozývaly podivné zvuky. Jednoho dne se odvážný rytíř rozhodl,",
    "Recept na palačinky:\n1. Smíchejte 250g hladké mouky a 2 vejce.\n2. Přidejte 500ml mléka a špetku soli.\n3.",
    "Na obloze se objevila duha. Měla všechny barvy:",
    "V roce 2050 bude svět vypadat úplně jinak. Auta budou létat a lidé budou",
    "Příčinou globálního oteplování je především"
]

In [11]:
print("="*40)
print("GENERAL TEXT GENERATION")
print("="*40)
for i, prompt in enumerate(general_prompts):
    print(f"\n--- PROMPT {i+1} ---")
    print_wrapped(f"PROMPT: {prompt}")
    generated_text = generate_from_prompt(prompt, max_new_chars=120, temperature=0.75)
    print_wrapped(f"MODEL: {generated_text}")

GENERAL TEXT GENERATION

--- PROMPT 1 ---
PROMPT: Byl jednou jeden starý hrad. V noci se z něj ozývaly podivné zvuky. Jednoho dne se odvážný
rytíř rozhodl,
MODEL: byl jednou jeden starý hrad. v noci se z něj ozývaly podivné zvuky. jednoho dne se odvážný
rytíř rozhodl,ž esyi ěoyvč eo h ola-u y-u ojd yue rn aza,kesvly   e aa. n 000 eeov  ercmc 09 anio eo
ec1 9 nb ee eo 9..n10 rk15  šrup

--- PROMPT 2 ---
PROMPT: Recept na palačinky: 1. Smíchejte 250g hladké mouky a 2 vejce. 2. Přidejte 500ml mléka a
špetku soli. 3.
MODEL: recept na palačinky 1. smíchejte 250g hladké mouky a 2 vejce. 2. přidejte 500ml mléka a
špetku soli. 3.pokdjaurv eibr ebt osaa oa eb et aegsrfeeu dnshnvld rdkmvdt nkbre yue rcene yese edt
soige ai etale yee crmnodt klb  vrt

--- PROMPT 3 ---
PROMPT: Na obloze se objevila duha. Měla všechny barvy:
MODEL: na obloze se objevila duha. měla všechny barvy  s íáksmrnmnmnn,ke tetv. oljšeoá áitozk 01 y i
erntt kaeats asrpi s etinsýi kěíae s osd  e uč s ýmvu enyv.ojsatj  eiu

---

## Part 1: In-Domain Question Answering

In [12]:
in_domain_questions = {
    "Hlavní město ČR": [
        "Otázka: Jaké je hlavní město České republiky?\nOdpověď:",
        "Hlavní město České republiky je",
        "Centrem politického a kulturního dění v Česku je město zvané"
    ],
    "Autor Babičky": [
        "Kdo je autorem knihy Babička?\nOdpověď:",
        "Dílo Babička napsala slavná česká spisovatelka",
        "Román o životě na Starém bělidle napsal(a)"
    ],
    "Skladatel Mé vlasti": [
        "Který skladatel složil cyklus symfonických básní Má vlast?\nOdpověď:",
        "Autorem Mé vlasti je",
        "Hudbu k Vltavě složil"
    ],
    "Karel IV.": [
        "Jaké bylo původní jméno Karla IV.?\nOdpověď:",
        "Otec vlasti, Karel IV., se původně jmenoval",
        "Když se narodil syn Elišky Přemyslovny, dostal jméno"
    ],
    "Nejvyšší hora ČR": [
        "Jak se jmenuje nejvyšší hora v České republice?\nOdpověď:",
        "Nejvyšší vrchol Česka je",
        "Vrchol, který v Krkonoších dosahuje výšky 1603 m n. m., se nazývá"
    ]
}


In [13]:
for topic, questions in in_domain_questions.items():
    print("="*40)
    print(f"TOPIC: {topic}")
    print("="*40)
    for q in questions:
        print_wrapped(f"PROMPT: {q}")
        generated_text = generate_from_prompt(q, max_new_chars=50, temperature=0.2)
        print_wrapped(f"MODEL: {generated_text}\n")
    print("\n")

TOPIC: Hlavní město ČR
PROMPT: Otázka: Jaké je hlavní město České republiky? Odpověď:
MODEL: otázka jaké je hlavní město české republiky? odpověďn aplk eie ee  ei ee e e e e e  e  e  e
e   e  e
PROMPT: Hlavní město České republiky je
MODEL: hlavní město české republiky jepia amr eret o ocrs  ee  ee ee ee ee ee ee ee e e
PROMPT: Centrem politického a kulturního dění v Česku je město zvané
MODEL: centrem politického a kulturního dění v česku je město zvanét oe
aataa."""""""""""""""""""""""""""""""""""""""


TOPIC: Autor Babičky
PROMPT: Kdo je autorem knihy Babička? Odpověď:
MODEL: kdo je autorem knihy babička? odpověďn as at. p ei ee e e e e  e    e e  e e     e e e
PROMPT: Dílo Babička napsala slavná česká spisovatelka
MODEL: dílo babička napsala slavná česká spisovatelkaasrcat eie eee ee  o ocrsu dl eiee ee ee e ee e
ee
PROMPT: Román o životě na Starém bělidle napsal(a)
MODEL: román o životě na starém bělidle napsalas  etea   o eie eee  o otik   oe ee  o oi ee ee  e


TOPIC: Skladatel 

## Part 3: Few-Shot Benchmarks

In [14]:
# <<< STEP 3: UPDATE THE BENCHMARK RUNNER >>>
def run_benchmark(benchmark_name, few_shot_prefix, benchmark_data, parse_function):
    """
    Runs a few-shot benchmark and prints the accuracy, using correct preprocessing.
    """
    correct_count = 0
    total_count = len(benchmark_data)
    results = []

    print("="*50)
    print(f"RUNNING BENCHMARK: {benchmark_name}")
    print("="*50)

    for question, true_answer in tqdm(benchmark_data.items()):
        prompt = few_shot_prefix + parse_function(question=question)
        full_output = generate_from_prompt(prompt, max_new_chars=25, temperature=0.1, top_k=1)
        
        # The prompt passed to the parse function must also be the cleaned one
        cleaned_prompt = clean_text(prompt)
        model_answer = parse_function(raw_output=full_output, prompt=cleaned_prompt)
        
        # --- THIS IS THE CRUCIAL NEW LINE ---
        # Clean the ground-truth answer to ensure a fair comparison
        cleaned_true_answer = clean_text(true_answer)
        # --- END NEW LINE ---

        # Compare answers. The model's output is already lowercase.
        is_correct = (model_answer == cleaned_true_answer)
        if is_correct:
            correct_count += 1
        
        results.append({
            "question": question, # Log the original question for readability
            "true_answer": cleaned_true_answer,
            "model_answer": model_answer,
            "is_correct": is_correct
        })

    # The rest of the function (displaying results) is the same...
    accuracy = (correct_count / total_count) * 100
    print(f"\n--- BENCHMARK '{benchmark_name}' COMPLETE ---")
    print(f"Accuracy: {accuracy:.2f}% ({correct_count} / {total_count})")

    # Display some examples
    print("\n--- Examples of Correct Answers ---")
    correct_examples = [r for r in results if r['is_correct']][:3]
    if not correct_examples: print("None in this run.")
    for ex in correct_examples: print(f"Q: {ex['question']} -> A: {ex['model_answer']} (Correct)")

    print("\n--- Examples of Incorrect Answers ---")
    incorrect_examples = [r for r in results if not r['is_correct']][:3]
    if not incorrect_examples: print("None in this run.")
    for ex in incorrect_examples: print(f"Q: {ex['question']} -> A: {ex['model_answer']} (Expected: {ex['true_answer']})\n")


## Benchmark 1: Factual Knowledge (Country -> Capital)

In [15]:
country_capital_prefix = "Německo -> Berlín\nPolsko -> Varšava\nUSA -> Washington, D.C.\n"
country_capital_data = {
    "Slovensko": "Bratislava", "Rakousko": "Vídeň", "Francie": "Paříž",
    "Itálie": "Řím", "Spojené království": "Londýn", "Japonsko": "Tokio",
    "Čína": "Peking", "Maďarsko": "Budapešť", "Španělsko": "Madrid",
    "Kanada": "Ottawa", "Austrálie": "Canberra", "Brazílie": "Brasília",
    "Indie": "Nové Dillí", "Egypt": "Káhira", "Finsko": "Helsinky"
}

In [16]:
def parse_country_capital(question=None, raw_output=None, prompt=None):
    if question:
        return f"{question} - "
    if raw_output:
        generated_part = raw_output[len(prompt):]
        return generated_part.split('\n')[0].strip()

In [17]:
run_benchmark(
    benchmark_name="Country -> Capital (Corrected)",
    few_shot_prefix=country_capital_prefix,
    benchmark_data=country_capital_data,
    parse_function=parse_country_capital
)

RUNNING BENCHMARK: Country -> Capital (Corrected)


  0%|          | 0/15 [00:00<?, ?it/s]


--- BENCHMARK 'Country -> Capital (Corrected)' COMPLETE ---
Accuracy: 0.00% (0 / 15)

--- Examples of Correct Answers ---
None in this run.

--- Examples of Incorrect Answers ---
Q: Slovensko -> A: vrhčc     er    e e    er (Expected: bratislava)

Q: Rakousko -> A: vrhasa      erkln    ekn (Expected: vídeň)

Q: Francie -> A: vrtlvr     ert   etal   e (Expected: paříž)



## Benchmark 2: Grammatical Skill (Singular -> Plural)

In [18]:
singular_plural_prefix = "pes (sg) -> psi (pl)\nžena (sg) -> ženy (pl)\nměsto (sg) -> města (pl)\n"
singular_plural_data = {
    "hrad": "hrady", "muž": "muži", "stroj": "stroje", "soudce": "soudci",
    "škola": "školy", "ulice": "ulice", "píseň": "písně", "kost": "kosti",
    "moře": "moře", "kuře": "kuřata", "nádraží": "nádraží", "světlo": "světla",
    "letadlo": "letadla", "auto": "auta", "náměstí": "náměstí"
}

In [19]:
def parse_singular_plural(question=None, raw_output=None, prompt=None):
    if question:
        return f"{question} sg - "
    if raw_output:
        generated_part = raw_output[len(prompt):]
        answer = generated_part.replace("pl", "").strip()
        return answer

In [20]:
run_benchmark(
    benchmark_name="Singular -> Plural (Corrected)",
    few_shot_prefix=singular_plural_prefix,
    benchmark_data=singular_plural_data,
    parse_function=parse_singular_plural
)

RUNNING BENCHMARK: Singular -> Plural (Corrected)


  0%|          | 0/15 [00:00<?, ?it/s]


--- BENCHMARK 'Singular -> Plural (Corrected)' COMPLETE ---
Accuracy: 0.00% (0 / 15)

--- Examples of Correct Answers ---
None in this run.

--- Examples of Incorrect Answers ---
Q: hrad -> A: žn e e  e   e   e  e   er (Expected: hrady)

Q: muž -> A: žn e e    er e e   e   e (Expected: muži)

Q: stroj -> A: žn e e  e   e   e   er e (Expected: stroje)

